## BAN5763 Group Exercise 3 - Question 1
Team: SFRR Analytics

In [1]:
# Import all the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from dotenv import load_dotenv
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, ChatMessage, AIMessage
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from PIL import Image, ImageDraw
import re
import tiktoken

## Document Similarity
1. **Document Loader** 
- This component is responsible for loading the documents like pdfs from their storage location.

2. **Data Chunking** 
- This task involves dividing the loaded documents into smaller, more manageable pieces or chunks. Chunking helps in processing large documents efficiently and can also improve the quality of embeddings by focusing on more coherent sections of text.

3. **Create embeddings and store in Chroma DB (vector store)** 
- This step involves generating numerical representations, or embeddings, of the document chunks using natural language processing techniques. These embeddings capture the semantic meaning of the text. The embeddings are then stored in a vector database, referred to as Chroma DB, which facilitates efficient similarity searches.

4. **Load existing Chroma DB (Optional)** 
- This optional step allows for the loading of an already existing Chroma DB. It is useful when you want to add new document embeddings to an existing database or simply utilize the previously stored embeddings for comparison without regenerating them.

5. **Calculate embedding cost** 
- This task calculates the computational or resource cost associated with generating embeddings. It may involve tracking the time, CPU usage, or memory consumption during the embedding process. This information is crucial for optimizing the system and managing resources effectively.

<br>

The `text-embedding-3-small` model is chosen for vector store embeddings because:

- Resource Efficiency: It consumes fewer computational resources, making it ideal for applications with limited processing power or when handling large datasets.
- Cost Effectiveness: The smaller model reduces operational costs, particularly in environments where computational resources are billed.
- Sufficient Accuracy: It offers adequate performance for many text similarity tasks without the need for the more nuanced understanding that larger models provide, balancing efficiency and effectiveness.

In [2]:
def load_document(file):
    """
    Load a document from a file and return the text content.
    """
    name, extension = os.path.splitext(file)

    if extension == '.pdf':
        from langchain.document_loaders import PyPDFLoader
        print(f'Loading {file}')
        loader = PyPDFLoader(file)
    elif extension == '.docx':
        from langchain.document_loaders import Docx2txtLoader
        print(f'Loading {file}')
        loader = Docx2txtLoader(file)
    elif extension == '.txt':
        from langchain.document_loaders import TextLoader
        loader = TextLoader(file)
    else:
        print('Document format is not supported!')
        return None

    data = loader.load()
    return data

# splitting data in chunks
def chunk_data(data, chunk_size=256, chunk_overlap=20):
    """
    Split the data into chunks of a given size and overlap.
    """
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = text_splitter.split_documents(data)
    print(f'Chunk Size: {chunk_size}, Number of chunks: {len(chunks)}')
    return chunks

# create embeddings and store in chroma db
def create_embeddings(chunks, embeddings):
    """
    Create embeddings for the given chunks and store them in a vector store.
    """
    
    # store in chroma db
    vector_store = Chroma.from_documents(chunks, embeddings, persist_directory='./mychroma_db')
    vector_store.persist()
    return vector_store

# load chroma db
def load_chroma_db(embeddings):
    """
    Load the vector store from the disk.
    """
    vector_store = Chroma(persist_directory="./mychroma_db", embedding_function=embeddings)
    return vector_store

# calculate embedding cost using tiktoken
def calculate_embedding_cost(texts):
    enc = tiktoken.encoding_for_model('text-embedding-3-small')
    embedding_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    print(f'Total Tokens: {embedding_tokens}')
    # text-embedding-3-small	$0.02 / 1M tokens
    print(f'Embedding Cost in USD: {embedding_tokens / 1000000 * 0.0002:.6f}')
    return embedding_tokens

In [3]:
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

embeddings = OpenAIEmbeddings(
    openai_api_key=OPENAI_API_KEY, 
    model='text-embedding-3-small'
    )

In [4]:
# load all pdfs in the directory
pdf_list = [f for f in os.listdir('.') if f.endswith('.pdf')]
load_pdf = False

# check if vector store file path exists
if os.path.exists('./mychroma_db'):
    # load the vector store from the disk
    vector_store = load_chroma_db(embeddings)
    # check if the vector store is loaded
    if vector_store._collection.count() > 0:
        print('Vector store loaded successfully')
    else:
        load_pdf = True
else:
    load_pdf = True

# load the pdf files if needed
if load_pdf:
    for file in pdf_list:
        # Load a PDF document and split it into sections
        data = load_document(file)

        # Split the data into chunks
        chunks = chunk_data(data, chunk_size=1024, chunk_overlap=256)

        # print the cost of the embeddings
        embedding_tokens = calculate_embedding_cost(chunks)

        # Create embeddings
        vector_store = create_embeddings(chunks, embeddings)

        print('###'*20)

    print(f'{vector_store._collection.count()} chunks of documents loaded in the vector store') 
else:
    print(f'Vector store already loaded with {vector_store._collection.count()} chunks of documents')

Loading Economy_of_China.pdf
Chunk Size: 1024, Number of chunks: 305
Total Tokens: 81643
Embedding Cost in USD: 0.000016
############################################################
Loading Economy_of_Germany.pdf
Chunk Size: 1024, Number of chunks: 128
Total Tokens: 36078
Embedding Cost in USD: 0.000007
############################################################
Loading Economy_of_India.pdf
Chunk Size: 1024, Number of chunks: 327
Total Tokens: 89935
Embedding Cost in USD: 0.000018
############################################################
Loading Economy_of_Japan.pdf
Chunk Size: 1024, Number of chunks: 162
Total Tokens: 46050
Embedding Cost in USD: 0.000009
############################################################
Loading Economy_of_the_United_Kingdom.pdf
Chunk Size: 1024, Number of chunks: 168
Total Tokens: 47598
Embedding Cost in USD: 0.000010
############################################################
Loading Economy_of_the_United_States.pdf
Chunk Size: 1024, Number of chunks

In [5]:
# print total number of documents in the vector store
print(f'Total number of chunks of documents in the vector store: {vector_store._collection.count()}')

Total number of chunks of documents in the vector store: 1391


In [20]:
# Example chunk 
vector_store.get('000bae14-0cfd-4984-afa3-498d2a7288c9')

{'ids': ['000bae14-0cfd-4984-afa3-498d2a7288c9'],
 'embeddings': None,
 'metadatas': [{'page': 11, 'source': 'Economy_of_Germany.pdf'}],
 'documents': ['32. "What Germany offers the world" (https://www.economist.com/news/\nbriefing/21552567-other-countries-would-love-import-germanys-eco\nnomic-model-its-way-doing-things). The Economist. 14 April 2012.\nArchived (https://web.archive.org/web/20180428020244/https://ww\nw.economist.com/news/briefing/21552567-other-countries-would-lo\nve-import-germanys-economic-model-its-way-doing-things) from the\noriginal on 28 April 2018. Retrieved 28 April 2018.\n33. "How Does Germany do It?" (https://www.asme.org/engineering-topi\ncs/articles/manufacturing-processing/how-does-germany-do-it).\nArchived (https://web.archive.org/web/20170902011441/https://ww\nw.asme.org/engineering-topics/articles/manufacturing-processing/h\now-does-germany-do-it) from the original on 2 September 2017.\nRetrieved 29 April 2017.\n34. Burger, Bruno (15 January 2020). "Publ

In [7]:
# Example Retrieved Document
query = 'biggest economy'
docs = vector_store.similarity_search_with_score(query, k=4)
for doc in docs:
    print(doc)

(Document(page_content='Largest Economy. We Shouldn\'t Be Shocked" (https://nationalintere\nst.org/feature/china-now-world%E2%80%99s-largest-economy-we-\nshouldn%E2%80%99t-be-shocked-170719). The National Interest.\nRetrieved 25 October 2020.\n131. Williamson, Peter J.; Hoenderop, Simon; Hoenderop, Jochem (3\nApril 2018). "An alternative benchmark for the validity of China\'s\nGDP growth statistics". Journal of Chinese Economic and Business\nStudies. 16 (2): 171–191. doi:10.1080/14765284.2018.1438867 (htt\nps://doi.org/10.1080%2F14765284.2018.1438867). ISSN 1476-\n5284 (https://www.worldcat.org/issn/1476-5284). S2CID 158464860\n(https://api.semanticscholar.org/CorpusID:158464860).', metadata={'page': 23, 'source': 'Economy_of_China.pdf'}), 0.8619288802146912)
(Document(page_content="unde r management had more than $30 trillion in assets.[62][63]\nDuring the Great Recession of 2008, the U.S. econom y suffered\na significant decline.[64][65] The American Reinvestment and\nRecovery Act wa

In [8]:
for i in range(len(docs)):
    print(f"""
Document metadata: {docs[i][0].metadata}
Document score: {docs[i][-1]}
Document content: {docs[i][0].page_content[:200]}...
""")


Document metadata: {'page': 23, 'source': 'Economy_of_China.pdf'}
Document score: 0.8619288802146912
Document content: Largest Economy. We Shouldn't Be Shocked" (https://nationalintere
st.org/feature/china-now-world%E2%80%99s-largest-economy-we-
shouldn%E2%80%99t-be-shocked-170719). The National Interest.
Retrieved 25...


Document metadata: {'page': 0, 'source': 'Economy_of_the_United_States.pdf'}
Document score: 0.8711493015289307
Document content: unde r management had more than $30 trillion in assets.[62][63]
During the Great Recession of 2008, the U.S. econom y suffered
a significant decline.[64][65] The American Reinvestment and
Recovery Act...


Document metadata: {'page': 0, 'source': 'Economy_of_the_United_States.pdf'}
Document score: 0.8777472972869873
Document content: econom y.[36][37][38] It is the world's largest econom y by
nominal GDP; it is also the second largest by purchasing power
parity (PPP), behind China.[39] It has the world's seventh
highest per capita...


Do

## QnA

1. **Message List Compiler**
- Retrieval from Vector Store: Retrieves the most relevant document chunks related to the user query, ensuring the AI has contextually appropriate information to generate accurate responses.
- Document Context: Assembles contextual data from the retrieved documents, which is then used to inform the AI's responses, enhancing relevance and accuracy.
- Chat History: Manually manage chat history to be considered in AI context. 
- Token Counting: Calculates token counts for the system and historical messages, which is essential for managing the token budget and understanding computational cost.

2. **Append to Chat History**
- This function updates the conversation history by appending the latest user query and AI response. 

3. **Getting AI Response**
- Message Handling: Constructs the complete message list needed for generating an AI response, facilitating a structured dialogue.
- Response Generation: Utilizes the chat model to generate responses based on the current conversation context and stored history.
- Cost Calculation: Estimates the cost of processing the user's query and the AI's response in terms of tokens, which is important for budgeting and system scaling.

<br>

The `gpt-4-0125-preview` model is chosen for QnA model because:
- Precision and Context Handling: The GPT-4 model is known for its high precision and ability to handle nuanced contexts, making it suitable for generating accurate and relevant responses in a Q&A system.
- Cost Efficiency: The 0125-preview variant strikes a balance between performance and cost, offering advanced capabilities of GPT-4 while being more cost-effective than larger models.
- Adaptability: This model variant is capable of adapting to diverse topics and user inquiries, making it versatile for a dynamic Q&A environment on complex topics like economics.

In [9]:
def num_tokens(text):
    enc = tiktoken.get_encoding("cl100k_base")
    enc = tiktoken.encoding_for_model('gpt-4-0125-preview')
    return len(enc.encode(text))

def message_list_info(user_query, vector_store):

    # Retrieval from the vector database, only get 2 most relevant chunks
    docs = vector_store.similarity_search(user_query, k=2)
    context = "\nFrom vector database: \n"
    
    for i in range(len(docs)):
        context = context + f"Retrieval document [{i}]: \n" + docs[i].page_content + "\n"

    # set the system message with the retrieved context from the vector database
    system_message = f'''You are an expert economist.

Your task is to answer questions about the top 6 largest economies in the world - USA, China, Germany, Japan, India, and UK.

WARNING: Your max response length is 200 tokens. So, please keep your responses concise.

Please refer to retrieved context (RAG) about the economy by country from Wikipedia:
{context}
'''
    message_list = [
        SystemMessage(content=system_message)
    ]

    system_message_token = num_tokens(system_message)

    # Load the conversation history from a CSV file
    df_history = pd.read_csv('static/' + 'df_history.csv')
    history_token = 0

    # Append the conversation history to the conversation history
    if len(df_history) < 10:
        for _, row in df_history.sort_index().iterrows():
        # for last 10 messages to reduce cost
            if row['entity'] == 'user':
                message_list.append(
                    HumanMessage(content=row['message'])
                )
            else:
                message_list.append(
                    AIMessage(content=row['message'])
                )
            history_token += num_tokens(row['message'])

    else:
        for _, row in df_history.sort_index().tail(10).iterrows():
            if row['entity'] == 'user':
                message_list.append(
                    HumanMessage(content=row['message'])
                )
            else:
                message_list.append(
                    AIMessage(content=row['message'])
                )
            history_token += num_tokens(row['message'])


    # Append the latest user query to the conversation history
    message_list.append(
        HumanMessage(content=user_query)
    )
    user_query_token = num_tokens(user_query)
    
    token_dict = {
        'system_message_token': system_message_token,
        'history_token': history_token,
        'user_query_token': user_query_token
    }

    return message_list, user_query, token_dict

def append_to_history(ai_response_text, user_query):
    csv_file = 'static/df_history.csv'
    df_history = pd.read_csv(csv_file)

    df_history = pd.concat([
        df_history,
        pd.DataFrame({
            'entity': ['user', 'ai_copilot'],
            'message':[user_query, ai_response_text]
        })
    ], ignore_index=True)
    df_history.to_csv(csv_file, index=False)


def get_ai_response(open_ai_chat_model, user_query, vector_store):
    # Get the message list
    message_list, user_query, token_dict = message_list_info(user_query, vector_store)

    # Get the AI response
    openai_response = open_ai_chat_model(messages=message_list)
    ai_response_text = openai_response.content

    # Save the conversation history to a CSV file
    append_to_history(ai_response_text, user_query)

    gpt4_cost = [10/1e6, 30/1e6] # input, output cost per token
    # print the tokens
    print(f"System message token: {token_dict['system_message_token']}")
    print(f"History message token: {token_dict['history_token']}")
    print(f"User query token: {token_dict['user_query_token']}")
    print(f"AI response token: {num_tokens(ai_response_text)}")
    # print the cost
    print(f"System message cost: {token_dict['system_message_token'] * gpt4_cost[0]:.6f}")
    print(f"History message cost: {token_dict['history_token'] * gpt4_cost[0]:.6f}")
    print(f"User query cost: {token_dict['user_query_token'] * gpt4_cost[0]:.6f}")
    print(f"AI response cost: {num_tokens(ai_response_text) * gpt4_cost[1]:.6f}")

    print("###"*20)
    print(f"User Query: {user_query}\nAI Response: {ai_response_text}")

    return ai_response_text

In [10]:
# create static folder
if not os.path.exists('static'):
    os.makedirs('static')
# create df for history and save to csv
df = pd.DataFrame(columns=['entity', 'message'])
df.to_csv('static/df_history.csv', index=False)

# Initialize the OpenAI chat model
openai_chat_model = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model='gpt-4-0125-preview',
    temperature=0.5,
    max_tokens=350,
)

c:\Users\roedj\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [11]:
query = "What is your goal?"

get_ai_response(openai_chat_model, query, vector_store)

c:\Users\roedj\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


System message token: 443
History message token: 0
User query token: 5
AI response token: 39
System message cost: 0.004430
History message cost: 0.000000
User query cost: 0.000050
AI response cost: 0.001170
############################################################
User Query: What is your goal?
AI Response: My goal is to provide concise and informative responses to questions regarding the economies of the top 6 largest economies in the world: USA, China, Germany, Japan, India, and the UK.


'My goal is to provide concise and informative responses to questions regarding the economies of the top 6 largest economies in the world: USA, China, Germany, Japan, India, and the UK.'

In [12]:
query = "What is the latest GDP of the United States?"

get_ai_response(openai_chat_model, query, vector_store)

System message token: 700
History message token: 44
User query token: 10
AI response token: 34
System message cost: 0.007000
History message cost: 0.000440
User query cost: 0.000100
AI response cost: 0.001020
############################################################
User Query: What is the latest GDP of the United States?
AI Response: The latest nominal GDP of the United States was $19.5 trillion in 2017, and it reached $20.1 trillion in Q1 2018.


'The latest nominal GDP of the United States was $19.5 trillion in 2017, and it reached $20.1 trillion in Q1 2018.'

In [13]:
query = "Can you explain the main industries driving China's economy?"

get_ai_response(openai_chat_model, query, vector_store)

System message token: 511
History message token: 88
User query token: 11
AI response token: 200
System message cost: 0.005110
History message cost: 0.000880
User query cost: 0.000110
AI response cost: 0.006000
############################################################
User Query: Can you explain the main industries driving China's economy?
AI Response: China's economy is driven by several key industries, including:

1. **Manufacturing**: As the world's manufacturing powerhouse, it produces a significant portion of global industrial products, including electronics, apparel, and machinery.
2. **Mining and Ore Processing**: Essential for its steel and aluminum production.
3. **Steel and Aluminium Production**: Vital materials for construction and manufacturing.
4. **Coal**: A major energy source.
5. **Textiles and Apparel**: Significant export goods.
6. **Petroleum and Chemicals**: Important for energy and various industrial processes.
7. **Food Processing**: A growing sector catering t

"China's economy is driven by several key industries, including:\n\n1. **Manufacturing**: As the world's manufacturing powerhouse, it produces a significant portion of global industrial products, including electronics, apparel, and machinery.\n2. **Mining and Ore Processing**: Essential for its steel and aluminum production.\n3. **Steel and Aluminium Production**: Vital materials for construction and manufacturing.\n4. **Coal**: A major energy source.\n5. **Textiles and Apparel**: Significant export goods.\n6. **Petroleum and Chemicals**: Important for energy and various industrial processes.\n7. **Food Processing**: A growing sector catering to both domestic and international markets.\n8. **Automobiles and Transportation Equipment**: Including cars, rail cars, ships, and aircraft.\n9. **Consumer Products**: Such as footwear, toys, and electronics.\n10. **Telecommunications and Information Technology**: A rapidly growing sector.\n\nThese industries reflect China's transition from a pri

In [14]:
query = "Based on the information you provided about China, how does Germany's industrial base differ?"

get_ai_response(openai_chat_model, query, vector_store)

System message token: 552
History message token: 299
User query token: 17
AI response token: 338
System message cost: 0.005520
History message cost: 0.002990
User query cost: 0.000170
AI response cost: 0.010140
############################################################
User Query: Based on the information you provided about China, how does Germany's industrial base differ?
AI Response: Germany's industrial base differs from China's in several key aspects:

1. **High-Quality Manufacturing**: Germany is renowned for its high-quality manufacturing, particularly in automotive, machinery, and chemical products. Brands like BMW, Siemens, and BASF are global leaders.

2. **Engineering and Innovation**: Germany places a strong emphasis on engineering excellence, innovation, and high-value-added manufacturing. It is a world leader in developing and manufacturing high-tech machinery and industrial equipment.

3. **Renewable Energy**: Germany is a pioneer in the renewable energy sector, particu

"Germany's industrial base differs from China's in several key aspects:\n\n1. **High-Quality Manufacturing**: Germany is renowned for its high-quality manufacturing, particularly in automotive, machinery, and chemical products. Brands like BMW, Siemens, and BASF are global leaders.\n\n2. **Engineering and Innovation**: Germany places a strong emphasis on engineering excellence, innovation, and high-value-added manufacturing. It is a world leader in developing and manufacturing high-tech machinery and industrial equipment.\n\n3. **Renewable Energy**: Germany is a pioneer in the renewable energy sector, particularly in wind power, and is committed to the Energiewende, transitioning to renewable energy. It leads in the production of wind turbines.\n\n4. **Chemical Industry**: While both countries have strong chemical industries, Germany's BASF is the world's largest chemical producer, focusing on a wide range of chemical products for various industries.\n\n5. **Automotive Industry**: Both

In [15]:
query = "What are the key challenges facing Japan's economy as per the latest data?"

get_ai_response(openai_chat_model, query, vector_store)

System message token: 661
History message token: 654
User query token: 15
AI response token: 173
System message cost: 0.006610
History message cost: 0.006540
User query cost: 0.000150
AI response cost: 0.005190
############################################################
User Query: What are the key challenges facing Japan's economy as per the latest data?
AI Response: Japan's economy faces several key challenges:

1. **High Levels of Public Debt**: With public debt at approximately 260% of GDP, Japan has the highest level of public debt among developed nations. A significant portion of this debt is held by the Bank of Japan.

2. **Aging and Declining Population**: Japan's population peaked in 2010 and has been declining since. Projections indicate it could fall below 100 million by the mid-21st century, impacting labor markets, consumption, and economic growth.

3. **Economic Growth**: Japan's GDP growth rates are modest, with projections of 1.3% in 2023 and 1.0% in 2024, indicating s

"Japan's economy faces several key challenges:\n\n1. **High Levels of Public Debt**: With public debt at approximately 260% of GDP, Japan has the highest level of public debt among developed nations. A significant portion of this debt is held by the Bank of Japan.\n\n2. **Aging and Declining Population**: Japan's population peaked in 2010 and has been declining since. Projections indicate it could fall below 100 million by the mid-21st century, impacting labor markets, consumption, and economic growth.\n\n3. **Economic Growth**: Japan's GDP growth rates are modest, with projections of 1.3% in 2023 and 1.0% in 2024, indicating slow economic expansion.\n\nThese challenges, coupled with the need for structural reforms in labor and markets, pose considerable obstacles to Japan's economic prospects."

In [16]:
query = "Could you update me on any major economic reforms in the UK mentioned in the documents, and discuss their potential impact?"

get_ai_response(openai_chat_model, query, vector_store)

System message token: 675
History message token: 842
User query token: 23
AI response token: 286
System message cost: 0.006750
History message cost: 0.008420
User query cost: 0.000230
AI response cost: 0.008580
############################################################
User Query: Could you update me on any major economic reforms in the UK mentioned in the documents, and discuss their potential impact?
AI Response: The documents mention the UK's response to the financial crisis and economic downturn around 2008-2009, including significant measures such as cutting interest rates to historic lows and initiating quantitative easing (QE) by the Bank of England (BoE). These actions aimed to boost lending, stimulate economic activity, and support the UK economy during a period of severe recession.

The potential impacts of these reforms include:

1. **Stimulated Economic Activity**: Lower interest rates made borrowing cheaper for businesses and consumers, encouraging investment and spendin

"The documents mention the UK's response to the financial crisis and economic downturn around 2008-2009, including significant measures such as cutting interest rates to historic lows and initiating quantitative easing (QE) by the Bank of England (BoE). These actions aimed to boost lending, stimulate economic activity, and support the UK economy during a period of severe recession.\n\nThe potential impacts of these reforms include:\n\n1. **Stimulated Economic Activity**: Lower interest rates made borrowing cheaper for businesses and consumers, encouraging investment and spending.\n\n2. **Increased Money Supply**: QE, involving the purchase of government securities, increased the money supply, further aiming to stimulate economic activity by encouraging lending and investment.\n\n3. **Inflationary Pressures**: While these measures can boost economic activity, they also carry the risk of leading to inflationary pressures, as an increased money supply can lead to higher prices if not matc

In [17]:
query = "Can you please explain the high economic growth rate of India in the last decade?"

get_ai_response(openai_chat_model, query, vector_store)

System message token: 614
History message token: 1107
User query token: 16
AI response token: 331
System message cost: 0.006140
History message cost: 0.011070
User query cost: 0.000160
AI response cost: 0.009930
############################################################
User Query: Can you please explain the high economic growth rate of India in the last decade?
AI Response: India's high economic growth rate in the last decade can be attributed to several factors:

1. **Economic Liberalization**: Since the 1990s, India has been progressively liberalizing its economy, reducing state control and making it easier for foreign businesses to invest and operate in India. This has spurred growth in sectors like services, IT, and manufacturing.

2. **Demographics**: India has a young population, with a median age significantly lower than that of other major economies. This demographic dividend has contributed to a growing labor force and increasing domestic consumption.

3. **Services Sector*

"India's high economic growth rate in the last decade can be attributed to several factors:\n\n1. **Economic Liberalization**: Since the 1990s, India has been progressively liberalizing its economy, reducing state control and making it easier for foreign businesses to invest and operate in India. This has spurred growth in sectors like services, IT, and manufacturing.\n\n2. **Demographics**: India has a young population, with a median age significantly lower than that of other major economies. This demographic dividend has contributed to a growing labor force and increasing domestic consumption.\n\n3. **Services Sector**: The services sector, especially IT and IT-enabled services, has been a significant driver of India's economic growth, contributing substantially to GDP, employment, and exports.\n\n4. **Urbanization**: Rapid urbanization has contributed to economic growth by creating new markets, increasing consumption, and attracting investment in urban infrastructure and real estate

In [18]:
df_history = pd.read_csv('static/' + 'df_history.csv')
df_history

,entity,message
0,user,What is your goal?
1,ai_copilot,My goal is to provide concise and informative ...
2,user,What is the latest GDP of the United States?
3,ai_copilot,The latest nominal GDP of the United States wa...
4,user,Can you explain the main industries driving Ch...
5,ai_copilot,China's economy is driven by several key indus...
6,user,Based on the information you provided about Ch...
7,ai_copilot,Germany's industrial base differs from China's...
8,user,What are the key challenges facing Japan's eco...
9,ai_copilot,Japan's economy faces several key challenges:\...
